<a href="https://colab.research.google.com/github/imtiajahmed318/Data-Analysis-with-python/blob/main/Laptop_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import re

In [7]:
df = pd.read_csv('/content/laptop_price - dataset.csv')
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,CPU_Company,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Company,GPU_Type,OpSys,Weight (kg),Price (Euro)
0,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,2.3,8,128GB SSD,Intel,Iris Plus Graphics 640,macOS,1.37,1339.69
1,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel,Core i5,1.8,8,128GB Flash Storage,Intel,HD Graphics 6000,macOS,1.34,898.94
2,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel,Core i5 7200U,2.5,8,256GB SSD,Intel,HD Graphics 620,No OS,1.86,575.00
3,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel,Core i7,2.7,16,512GB SSD,AMD,Radeon Pro 455,macOS,1.83,2537.45
4,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel,Core i5,3.1,8,256GB SSD,Intel,Iris Plus Graphics 650,macOS,1.37,1803.60


In [8]:
df.describe()

,Inches,CPU_Frequency (GHz),RAM (GB),Weight (kg),Price (Euro)
count,1275.000000,1275.000000,1275.000000,1275.000000,1275.000000
mean,15.022902,2.302980,8.440784,2.040525,1134.969059
std,1.429470,0.503846,5.097809,0.669196,700.752504
min,10.100000,0.900000,2.000000,0.690000,174.000000
25%,14.000000,2.000000,4.000000,1.500000,609.000000
50%,15.600000,2.500000,8.000000,2.040000,989.000000
75%,15.600000,2.700000,8.000000,2.310000,1496.500000
max,18.400000,3.600000,64.000000,4.700000,6099.000000


In [9]:
df.isnull().value_counts()

,,,,,,,,,,,,,,,count
Company,Product,TypeName,Inches,ScreenResolution,CPU_Company,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Company,GPU_Type,OpSys,Weight (kg),Price (Euro),
False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1275


In [10]:
df.isnull().sum()

,0
Company,0
Product,0
TypeName,0
Inches,0
ScreenResolution,0
CPU_Company,0
CPU_Type,0
CPU_Frequency (GHz),0
RAM (GB),0
Memory,0


In [14]:

label_encoder = LabelEncoder()

columns_to_label = ['Company', 'CPU_Company', 'CPU_Type', 'Memory', 'GPU_Company', 'GPU_Type', 'OpSys']

for column in columns_to_label:
    df[column] = df[column].astype(str)
    df[column] = label_encoder.fit_transform(df[column])

df = df.drop(columns=['Product','TypeName'])
df.head()

,Company,Inches,ScreenResolution,CPU_Company,CPU_Type,CPU_Frequency (GHz),RAM (GB),Memory,GPU_Company,GPU_Type,OpSys,Weight (kg),Price (Euro)
0,1,13.3,IPS Panel Retina Display 2560x1600,1,40,2.3,8,4,2,56,8,1.37,1339.69
1,1,13.3,1440x900,1,40,1.8,8,2,2,50,8,1.34,898.94
2,7,15.6,Full HD 1920x1080,1,46,2.5,8,16,2,52,4,1.86,575.00
3,1,15.4,IPS Panel Retina Display 2880x1800,1,54,2.7,16,29,0,76,8,1.83,2537.45
4,1,13.3,IPS Panel Retina Display 2560x1600,1,40,3.1,8,16,2,57,8,1.37,1803.60


In [16]:
from sklearn.model_selection import train_test_split

X = df.drop('Price (Euro)', axis=1)
y = df['Price (Euro)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X=df
# .drop(columns=df.columns[12])
# y=df[df.columns[12]]
X_scaled = pd.DataFrame(X.drop(columns=['Company','CPU_Company','CPU_Type','Memory','GPU_Company','GPU_Type','OpSys']))
# print(X_scaled.head())


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_scaled)
# y = scaler.fit_transform(y)


X_scaled = pd.DataFrame(X_scaled, columns=['Inches','ScreenResolution','CPU_Frequency (GHz)','RAM (GB)','Weight (kg)', 'Price (Euro)'])
print(X_scaled.head())
X = pd.concat([X[['Company','CPU_Company','CPU_Type','Memory','GPU_Company','GPU_Type','OpSys']], X_scaled], axis=1)
y=X[X.columns[12]]
X=X.drop(columns=df.columns[12])
X.head()

ValueError: could not convert string to float: 'IPS Panel Retina Display 2560x1600'